# 第 9 章：硬件相关数学（Hardware-Aware Math: Roofline, Tiling, SIMD, FMA）

本章聚焦：

- Roofline 模型：算力 vs 带宽
- 算术强度（arithmetic intensity）
- 分块（tiling）与数据重用
- SIMD / FMA / GEMM 内核的数学视角

部署 = 数学 × 数值稳定性 × 结构优化 × 硬件利用率
本章进入“接近底层硬件”的数学，解释为什么你的模型在 GPU/NPU/CPU 上表现完全不同，以及编译器/内核为什么要做 tiling、packing、fusion。

---


## 9.1 Roofline 模型

Roofline 模型给出在给定硬件上，某个 kernel 的理论性能上界：

$$
P_{\text{attainable}} = \min\left(P_{\text{peak}},\ I \cdot B\right)
$$


| 量                 | 含义            | 单位            | 举例                     |
| ----------------- | ------------- | ------------- | ---------------------- |
| $P_{\text{peak}}$ | 峰值算力（FLOPs/s） | TFLOPs / TOPS | GPU: 400 TFLOPs        |
| $B$               | DRAM 带宽       | GB/s          | GPU: 1200 GB/s         |
| $I$               | 算术强度          | FLOPs/byte    | Conv: 高；elementwise: 低 |


算术强度定义为：

$$
I = \frac{\text{FLOPs}}{\text{Bytes moved}}
$$

### 9.1.1 Roofline 的两种 regime

1. **memory-bound**（带宽受限）：
   - 若 $I \cdot B < P_{\text{peak}}$
   - 性能主要受内存带宽限制，FLOPs 很充裕
2. **compute-bound**（算力受限）：
   - 若 $I \cdot B \ge P_{\text{peak}}$
   - 性能由算力上限决定

- 典型 memory-bound 算子：

| 算子                            | 原因                 |
| ----------------------------- | ------------------ |
| ReLU / GELU / Add / LayerNorm | 每元素只有 1–3 次计算，但要访存 |
| 变形/transpose 类算子              | 全是搬运数据             |
| Embedding lookup              | 几乎是纯读内存            |

→ 工程手段：fusion、tiling、减少内存往返、cache-friendly layout

- 典型 compute-bound 算子：

| 算子               | 原因      |
| ---------------- | ------- |
| GEMM / Conv      | 乘加密集    |
| Attention matmul | 高 FLOPs |

→ 工程手段：使用最佳内核、tensor core、int8/int4、tiling

**【工程视角】**  

- 对算子而言：
  - Conv/GEMM 通常具有较高 $I$，更容易成为 compute-bound
  - 简单 element-wise op 的 $I$ 很低，通常是 memory-bound
- 对 Embedding 而言：几乎没有 FLOPs，但读写大量数据 → 典型 memory-bound

---

### 9.1.2 算术强度差异可大到几个数量级
示例（FP16）：
| 操作               | FLOPs | Byte   | I = FLOPs/Byte |
| ---------------- | ----- | ------ | -------------- |
| `x = x + y`      | 1     | 8      | 0.125          |
| `GEMM 1024×1024` | 2·10⁹ | ~32 MB | ~60            |
| `Conv 3×3, 64ch` | 1000+ | 64     | >15            |

解释：

Conv/GEMM 一次加载数据可重复使用多次，因此算术强度高。

而 elementwise op 没有数据重用。
→ fusion 对 LN/GELU/Activation 有巨大收益，而对 GEMM 几乎没意义。

## 9.2 分块（Tiling）与数据重用

以 GEMM 为例：$C = A B$，其中：

- $A \in \mathbb{R}^{M \times K}$
- $B \in \mathbb{R}^{K \times N}$
- $C \in \mathbb{R}^{M \times N}$

如果直接三重循环，会频繁从内存加载 A/B/C，导致带宽浪费。
```python
for (i = 0; i < M; ++i)
  for (j = 0; j < N; ++j)
    for (k = 0; k < K; ++k)
        C[i][j] += A[i][k] * B[k][j];
```
原因：假设 $M=N=K= 2048$
- 矩阵大小约 ~16MB（FP32），肯定 放不进 L1/L2 cache。
- 每次访问 B[k][j] 都是在跨行跳跃（Cache line 效率极低）B[0][0], B[1][0], B[2][0], ...
- C[i][j] 也不断被读/写到 DRAM
- 数据重用非常差. 某行 A[i][:] 会被重复使用 M 次, 但 naive 代码里：

    - 内层 k 改变得太快

    - cache line 刚读进来就被挤掉

    - 重用不发生在 cache 里，而是再次从 DRAM load
    
    - B 的重用：每列 B[:][j] 理论上应重复用 N 次. 但 naive 访问 B 是跨行 → cache line 完全白费。
- 结果：几乎所有 load 都来自 DRAM

分块思想：

- 将 C 分成小块（tile）：$M_b \times N_b$
- 对每个 tile：
  - 加载对应的 A 子块（$M_b \times K_b$）
  - 加载对应的 B 子块（$K_b \times N_b$）
  - 在寄存器或小缓存中完成所有 FMA

约束条件：

$$
(M_b \cdot K_b + K_b \cdot N_b + M_b \cdot N_b) \cdot \text{bytes} \le \text{cache size}
$$

关键逻辑：
一个 tile 在 cache/寄存器中反复使用，使得数据复用最大化。
访问一次 DRAM → 用几十次、上百次。

**【直观理解】**  

- 通过分块，让同一块数据被多次重用（reuse），有效提高算术强度 $I$
- 减少对 DRAM 的访问次数

---

### 9.2.2 为什么 tiling 是所有编译器/内核的“母操作”？


- PyTorch Inductor
- TensorRT
- ONNX Runtime
- CUTLASS/CUBLAS
- TVM
- Apple ANE 内核
- Qualcomm Hexagon DSP
- NVIDIA NPU

都会一定进行：Tiling → Vectorization → Reuse → FMA/TensorCore mapping
在 runtime inference 中：

- tiling 大小 = 软件决定

- tile 下沉到 tensor core → 必须是硬件决定
（如 16×16、32×8、32×32）
---


## 9.3 SIMD / FMA / Tensor Core

### 9.3.1 FMA 再回顾

$$
\text{FMA}(a,b,c) = a \times b + c
$$


- 在硬件中可以作为一个基本指令
- dot product 与 GEMM 内部都是大量 FMA
- 精度更好（融合运算只做一次 rounding）

矩阵乘内部全是 FMA：
$$C_{ij} += A_{ik}\cdot B_{kj}$$

### 9.3.2 SIMD（Single Instruction, Multiple Data）

SIMD 指令可以：

- 一条指令对多个数据元素执行相同运算
- 例如 
   - AVX2: 256-bit → 8 × FP32

   - AVX512: 512-bit → 16 × FP32

   - ARM NEON: 128-bit

   - Apple AMX/Tensor Core: tile-based SIMD

矩阵乘法内部：

- 每次从内存加载一个向量 block
- 使用 SIMD/FMA 对多个元素并行乘加

### 9.3.3 Tensor Core / Matrix Unit

在现代 GPU/NPU 中：

- 提供专门的矩阵乘单元（如 NVIDIA Tensor Core、Intel AMX）
- 每条指令执行一个小矩阵乘运算（如 16×16 × 16×16）

**【数学视角】**  

- 这些专用单元本质上实现了一个：

$$
C_{\text{tile}} = A_{\text{tile}} B_{\text{tile}} + C_{\text{tile}}
$$


- 你的任务是：
  - 把大矩阵拆成符合硬件 tile 大小的小块
  - 做好数据布局，使得这些 tile 在内存中连续、对齐

---

## 9.4 硬件意识下的量化与打包（Packing）

对 INT8 / INT4 GEMM：

- 权重/激活不仅被量化，还会按特定 pattern 打包（pack）到寄存器友好的格式
- 例如把多个 int4 压在一个 16-bit 或 32-bit 容器中

这涉及：

- 位运算（bitwise ops）
- 对齐（alignment）约束
- 多个元素的“并行 unpack”数学

虽然这里不展开细节，但要知道：

> 高性能量化推理内核的数学本质是：  
> **用整数算术和位运算实现“向量化 FMA”的等价行为。**
整个过程实现的是：
$$\sum_{k=1}^n w_k x_k$$

但硬件看到的是：

- 一堆 32-bit load

- 一堆 bitwise

- 一堆 dot-product-like 指令（如 DP4A）

### 9.4.1 DP4A 是量化内核的数学来源
DP4A：一次指令做 4 个 int8 dot products
$$result = \sum_{i=0}^{3} a_i b_i$$
INT8 GEMM 实际上是：

把所有 dot product 尽可能映射到 DP4A/Tensor Core int8 MMA

### 9.4.2 常见打包格式示例
| 格式                      | 描述                         |
| ----------------------- | -------------------------- |
| NCHW16c                 | 通道 16 对齐 → vector friendly |
| weight packing for GEMM | K blocking into 32         |
| int4 → int32 pack       | 连续 8 个 int4 合并到 1 个 32-bit |
---



## 9.5 Nvidia Thor Hardware-Aware Math
NVIDIA Thor（Drive Thor） 是 2025–2026 车端部署的旗舰 SoC，用于 L2++/L4 自动驾驶与多任务推理。
> 其核心意义：把高吞吐 Tensor Core、GPU SM、Deep Learning Accelerator（DLA）、高带宽内存整合在一个汽车级芯片上。

Thor 的关键指标
| 项           | 大致规格（公开）                    |
| ----------- | --------------------------- |
| 峰值算力（AI 推理） | ~800–1000+ TOPS（INT8）       |
| Tensor Core | 新一代、支持 FP8 / INT8 / INT4    |
| GPU         | Ada/Lovelace 世代             |
| 内存          | LPDDR5/X，带宽约数百 GB/s         |
| 功耗          | < 250W                      |
| 生态          | TensorRT、DriveOS、CUDA、cuDLA |

对部署工程师而言：Thor = 有强大 TensorCore 的 edge device
算力大、带宽中等、功耗有限、热容量有限 → 非常依赖 tiling、packing、quantization、compute-bound ↔ memory-bound 判断。

---

9.5.1 Thor 的 Roofline 分析
1. compute roof（算力上限）
如果 Thor INT8 = 1000 TOPS：
$$P_{\text{peak}} = 10^{15}\text{ ops/s}$$

2. 内存带宽
假设 LPDDR5X ≈ 200–300 GB/s（合理估计）：
$$B \approx 2.5\times10^{11} \text{ bytes/s}$$

3. 计算“分界线算术强度” I*：
$$I^*=\frac{P_{\text{peak}}}{B}
=\frac{10^{15}}{2.5\times10^{11}}\approx4000$$

结论：
算术强度低于 4000 FLOPs/Byte 的算子全部 memory-bound。

| 算子                  | I     | 判断               |
| ------------------- | ----- | ---------------- |
| ReLU / Add / GELU   | ~1–5  | memory-bound     |
| LN（W=768）           | ~10   | memory-bound     |
| Embedding           | ~0    | memory-bound     |
| Attention Softmax   | < 10  | memory-bound     |
| Conv 3×3 / GEMM 大矩阵 | 数百〜数千 | 接近 compute-bound |

→ Thor 的大多数 Transformer 非矩阵算子全是 memory-bound
→ 所以 fusion、tiling、weight packing 对 Thor 至关重要。

9.5.2 Thor 上的 Tiling / Tensor Core 映射
Thor 的 Tensor Core 可以执行：

- FP16 / FP8 / INT8 / INT4

- 小 tile（如 16×16 / 32×8 / etc.）矩阵乘

- 类似 Hopper/Lovelace 的 MMA（matrix-multiply-accumulate）
为了吃满 Tensor Core：
需要满足 3 个条件：
---
(1) tile 大小必须匹配 MMA 固定尺寸

例如 INT8 核心是：

- 16×16×32

- 32×8×16

- 等等（依 GPU gen变化）
你的 GEMM 必须拆成这些 block：
$$A_{tile}\in\mathbb{R}^{16\times32},\ 
B_{tile}\in\mathbb{R}^{32\times16},\ 
C_{tile}\in\mathbb{R}^{16\times16}$$
若维度不能整除，会带来 padding overhead。
---
(2) weight/activation 必须 pack 成 TensorCore-friendly 格式
Thor 上最典型布局：

- INT8：K 方向 block 32 对齐， K = 内积维度（inner dimension / reduction dimension） $C_{M \times N} = A_{M \times K} \times B_{K \times N}$

- INT4：K 方向 block 64 对齐

- 权重量化后还要 re-order 成 warp-friendly 格式
否则：性能可能下降 5–20×。
---
(3) Tiling 必须足够大，避免 memory-bound

因为 I*≈4000 FLOPs/byte 非常高，如果 tile 太小：

- 数据重用少

- 算术强度低

- 直接掉到 memory-bound 区域

- 不可能吃满 1000 TOPS

### 9.5.3 Thor 上的 Transformer 推理：算子分类

典型 L2+/L4 自动驾驶模型包含：

- 感知（视觉 backbone + BEV）

- 融合（点云/BEV 融合）

- Transformer（BEVFormer、DETR3D、Plato、OCC 等）

- 预测

- 规划

- 控制网络

这些算子在 Thor 上的 Roofline 表现：

(1) 大 GEMM (Multi-Head Attention Q/K/V、FFN)

- compute-bound 或接近上限

- 只要维度够大：hidden=1024/2048 就能占满 Tensor Core

工程应用：

- 用 INT8/FP8 weights

- K 维度必须 32/64 对齐（依硬件）

- 采用 weight packing（W8A8/W4A8）

(2) Softmax + LayerNorm + GELU → memory-bound

无法吃算力：

- 访存次数 = O(N)

- FLOPs = 常数级

工程应用：

- 最好 fusion：LNorm + GELU + residual

- 或者切片 tiling（利用 L2 cache 提升算术强度）

- 或者使用 “FlashAttention” (DRAM I/O 降低)

(3) BEV/2D/3D 插值、reshape、concat → 严重 memory-bound

工程应用：

- layout 优化（NCHW → NHWC16）

- instrument L2 cache

- fuse transpose + copy

- 避免 Strided access

(4) CNN backbone / Conv2d → compute-bound（FP16 / INT8）

Conv 本身数据重用很高
→ 非常适合 Thor TensorCore

Thor 上的 Roofline 映射总结图
| 算子类型                     | 算术强度 (Ops/Byte)       | Thor 区域             | 瓶颈                  | 优化手段                                      |
|------------------------------|----------------------------|------------------------|------------------------|-----------------------------------------------|
| Large GEMM (Block > 128)     |  High (> 4000)             | Compute Plateau       | Tensor Core TOPS      | WGMMA, FP8, Pipeline                          |
| Small GEMM (Batch=1)         | Medium (~100-500)         | Transition            | Latency / Bandwidth   | Batching, Persistent Kernel                   |
| Conv2d (ResNet)              | High                      | Compute Plateau       | Tensor Core TOPS      | NHWC Layout                                   |
| Depthwise Conv               | Low                       | Memory Slope          | Bandwidth             | Kernel Fusion                                 |
| LayerNorm / Activation       | Very Low (< 10)           | Memory Slope          | Bandwidth             | Fused Kernels (e.g. TRT Plugin)               |
| BEV Grid Sample              | Very Low + High Latency   | Deep Memory Bound     | Cache Miss Rate       | Tiling, L2 Persistence, Shared Mem Pre-fetch  |

## 9.6 本章小结
部署需要完成以下判断：
1. 这个算子的 arithmetic intensity 是高还是低？

    - GEMM / Conv → 高 → compute-bound → 用最佳内核、tensor core、int8

    - LN / GELU / ReLU → 低 → memory-bound → fuse, reorder, reduce bandwidth

    - Embedding → 超低 → 完全看带宽 → layout + cache-friendly

2. 是否需要 tiling？如何选择 tile size？
你的推理应带着数学约束：
$$(M_bK_b+K_bN_b+M_bN_b)\cdot bytes \le cache$$

tile 越大：
- 算术强度高
- 但可能对齐/缓存不够

tile 越小：

- DRAM 来回更多
- 算术强度降低

3. 是否能利用 SIMD/FMA/Tensor Core？
    - weight layout 是否对齐？

    - activation 是否 contiguous？

    - tile 大小是否满足硬件 MMA 大小？

4. 是否能使用量化（INT8/INT4）与打包？
    - 带宽 ↓ 4x

    - tile reuse ↑

    - DP4A/TensorCore int8 ↑

    - 算术强度 I ↑

    - 性能 ↑ 2–6×（视硬件）-

5. 对整个模型性能的“Roofline 定位”
把所有算子画在 Roofline 上：

    - GEMM/Conv：靠顶部（compute-bound）

    - LN/GELU/Activation：靠左侧下方（memory-bound）

    - Embedding：最左侧最低（pure bandwidth）

这可以指导：

- fusion priority

- tiling priority

- quantization priority

- kernel scheduling

- 算子 placement（CPU/NPU/GPU）

> 到这里为止，你已经拥有了一套：  
> **从线性代数 → 优化 → 近似 → 统计 → 信息论 → 信号处理 → 图论 → 数值分析 → 硬件数学** 的完整思维链条。

在实际工程中，每一次做部署决策，你都可以在脑中快速走一遍这条链：

- 这个算子本质是什么线性代数？
- 能不能用压缩/近似/剪枝优化？
- 对分布/信息量有何影响？
- 会不会引发数值问题？
- 在硬件上是 compute-bound 还是 memory-bound？
- 该如何做 tiling / fusion / placement？